In [6]:
# download the dataset and save it in the same dir with this file.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel,RationalQuadratic,Exponentiation
from sklearn.tree import DecisionTreeRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn import neighbors
import os
kf = KFold(n_splits=10)
# import scikit learn
from sklearn import linear_model,svm
from sklearn.preprocessing import PolynomialFeatures

def getxandy(filename):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    y = df.z.to_numpy()
  return x,y

def getpolyxandy(filename,poly=2,withy=True):
  with open(filename,'r') as f:
    df = pd.read_csv(f,index_col=0)
    x = np.zeros((len(df.x),2))
    y = np.zeros((len(df.x)))
    x[:,0] = df.x.to_numpy()
    x[:,1] = df.y.to_numpy()
    if withy:
      y = df.z.to_numpy()
    else:
      y = None
    poly = PolynomialFeatures(degree=poly)
    x = poly.fit_transform(x)
  return x,y

def scoreperdataset(y_pred,y_test):
  return np.sqrt(np.sum(np.square(y_pred - y_test))/len(y_pred))

def writeresults(X_test,y_test,y_pred,f):
    idx = np.argsort(X_test[:,0])
    X_test = X_test[idx]
    y_test = y_test[idx]
    y_pred = y_pred[idx]
    data = {}
    data['x'] = X_test[:,0]
    data['y'] = X_test[:,1]
    data['z_test'] = y_test[idx]
    data['z_pred'] = y_pred[idx]
    df = pd.DataFrame(data)
    if not os.path.isdir('results/'):
      os.mkdir('results')
    df.to_csv('results/'+f+'.csv',float_format="%.6f")

In [ ]:
def getweights(l):
    npl = np.array(l)
    return np.square(1. / npl)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets[9:]:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(neighbors.KNeighborsRegressor(weights=getweights), {'n_neighbors':[x for x in range(1,150)]},n_jobs=30)
  #clf = GridSearchCV(neighbors.RadiusNeighborsRegressor(weights='uniform'), {'radius':[0.01*x for x in range(50,100)]},n_jobs=30)
  #clf = GridSearchCV(neighbors.RadiusNeighborsRegressor(weights='distance'), {'radius':[0.01*x for x in range(1,10)]},n_jobs=30)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.n_neighbors))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
#outdf.to_csv('submission_knn_small.csv',index_label='id')

In [4]:
import tensorflow as tf
import keras
import tensorflow as tf
tf.config.list_physical_devices('GPU')
#config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 20} ) 
#sess = tf.Session(config=config) 
#keras.backend.set_session(sess)

[]

In [3]:
import os
os.environ["LD_LIBRARY_PATH"] = '/lib64:/home/yhong/hpc_sdk/Linux_x86_64/20.9/cuda/10.1/lib64:/lib:/home/yhong/yuxibuild/anaconda3/lib'

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import r2_score
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy

trainfolder = 'small_datasets/'
polydegree=4
X,y = getpolyxandy(trainfolder+'J'+'/train.csv',polydegree)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)

import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import keras
# config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 20} ) 
# sess = tf.Session(config=config) 
# keras.backend.set_session(sess)
# create model
model = Sequential()
model.add(Dense(40, activation="relu", input_dim=X.shape[1], kernel_initializer="uniform"))
model.add(Dense(20, activation="relu", input_dim=40, kernel_initializer="uniform"))
model.add(Dense(10, activation="relu", input_dim=20, kernel_initializer="uniform"))
model.add(Dense(1, activation="linear", kernel_initializer="uniform"))

# Compile model
model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, epochs=800, batch_size=50,validation_split=0.2,verbose=2)

och 87/800
2592/2592 - 2s - loss: 0.9327 - accuracy: 0.0000e+00 - val_loss: 0.9611 - val_accuracy: 0.0000e+00
Epoch 88/800
2592/2592 - 2s - loss: 0.9301 - accuracy: 0.0000e+00 - val_loss: 0.9341 - val_accuracy: 0.0000e+00
Epoch 89/800
2592/2592 - 2s - loss: 0.9286 - accuracy: 0.0000e+00 - val_loss: 0.9111 - val_accuracy: 0.0000e+00
Epoch 90/800
2592/2592 - 2s - loss: 0.9277 - accuracy: 0.0000e+00 - val_loss: 0.9129 - val_accuracy: 0.0000e+00
Epoch 91/800
2592/2592 - 2s - loss: 0.9287 - accuracy: 0.0000e+00 - val_loss: 0.9023 - val_accuracy: 0.0000e+00
Epoch 92/800
2592/2592 - 2s - loss: 0.9253 - accuracy: 0.0000e+00 - val_loss: 0.9388 - val_accuracy: 0.0000e+00
Epoch 93/800
2592/2592 - 2s - loss: 0.9271 - accuracy: 0.0000e+00 - val_loss: 0.9012 - val_accuracy: 0.0000e+00
Epoch 94/800
2592/2592 - 2s - loss: 0.9256 - accuracy: 0.0000e+00 - val_loss: 0.9129 - val_accuracy: 0.0000e+00
Epoch 95/800
2592/2592 - 2s - loss: 0.9241 - accuracy: 0.0000e+00 - val_loss: 0.9077 - val_accuracy: 0.000

KeyboardInterrupt: 

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(neighbors.KNeighborsRegressor(weights='distance'), {'n_neighbors':[x for x in range(1,150)]},n_jobs=30)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.n_neighbors))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_knn_small.csv',index_label='id')

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(neighbors.KNeighborsRegressor(weights='distance'), {'n_neighbors':[x for x in range(1,150)]},n_jobs=30)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.n_neighbors))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_knn_small.csv',index_label='id')

In [ ]:
# pre 9
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[:9]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  #X,y = getxandy(trainfolder+'/'+d+'/train.csv')
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #break
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

In [ ]:
# pre 9 large

#model = Pipeline([('poly', PolynomialFeatures(degree=4)),('linear', DecisionTreeRegressor(max_depth=25))])

finalacc_large = []
trainfolder = 'large_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_large = scoreperdataset(y_pred, y_test)
  finalacc_large.append(loss)
  print("{} loss {:.5f}".format(d, loss_large))
  #writeresults(X_test,y_test,y_pred)
print('final average loss is {:.5f}'.format( sum(finalacc_large)/len(finalacc_large) ) )

In [ ]:
# pre 9
finalacc_small = []
trainfolder = 'small_datasets'
datasets = [chr(ord('A') + x) for x in range(18)]
for d in datasets[9:]:
  X,y = getpolyxandy(trainfolder+'/'+ d +'/train.csv',4)
  reg = DecisionTreeRegressor(max_depth=25)
  X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1)
  reg.fit(X_train,y_train)
  y_pred = reg.predict(X_test)
  loss_small = scoreperdataset(y_pred, y_test)
  finalacc_small.append(loss_small)
  print("{} loss {:.5f} ".format(d, loss_small))
  #writeresults(X_test,y_test,y_pred,f[9:])
print('final avg loss is {:.5f}'.format(sum(finalacc_small)/len(finalacc_small)))

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'small_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(DecisionTreeRegressor(), {'max_depth':[10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]},n_jobs=40)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.max_depth))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_{}_pd{}.csv'.format(trainfolder,polydegree),index_label='id')

In [ ]:
outdf

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
final_acc = []
pred_array = {}
trainfolder = 'large_datasets/'
datasets = [chr(ord('A') + x) for x in range(18)]
polydegree=4
for d in datasets:
  X,y = getpolyxandy(trainfolder+d+'/train.csv',polydegree)
  clf = GridSearchCV(DecisionTreeRegressor(), {'max_depth':[x for x in range(10,40)]},n_jobs=20)
  X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state=1)
  clf.fit(X_train, y_train)
  y_pred = clf.best_estimator_.predict(X_test)
  loss = scoreperdataset(y_pred,y_test)
  final_acc.append(loss)
  X_real,y_real = getpolyxandy(trainfolder+d+'/test.csv',polydegree,withy=False)
  y_real_pred = clf.best_estimator_.predict(X_real)
  pred_array[d+'.z'] = y_real_pred
  print('loss for {} is {}, maxdepth is {}'.format(d,loss, clf.best_estimator_.max_depth))
print('final avg loss is {:.5f}'.format( sum(final_acc)/len(final_acc)))
outdf = pd.DataFrame(pred_array)
outdf.to_csv('submission_{}_pd{}.csv'.format(trainfolder,polydegree),index_label='id')